In [1]:
#This is based on https://github.com/aniruddhraghu/sepsisrl/blob/master/preprocessing/process_interventions.ipynb

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame
import copy

In [4]:
orig_data = pd.read_csv("../Matlab/MIMIC_mimiciii.csv")

In [6]:
interventions = orig_data[["max_dose_vaso", "input_4hourly"]]

In [7]:
adjusted_vaso = interventions["max_dose_vaso"][interventions["max_dose_vaso"] >0]
adjusted_iv = interventions["input_4hourly"][interventions["input_4hourly"]>0]

In [8]:
vaso_quartiles = adjusted_vaso.quantile([0.25,0.50,0.75])
iv_quartiles = adjusted_iv.quantile([0.25,0.5,0.75])

In [9]:
vq = np.array(vaso_quartiles)

array([0.08, 0.2 , 0.45])

In [10]:
ivq = np.array(iv_quartiles)

array([ 48.2670625, 150.       , 500.       ])

In [11]:
discretised_int = copy.deepcopy(interventions)

In [12]:
discretised_int['vaso_input'] = discretised_int['max_dose_vaso']
discretised_int['vaso_input'][interventions['max_dose_vaso'] == 0.0] = 0
discretised_int['vaso_input'][(interventions['max_dose_vaso'] > 0.0) & (interventions['max_dose_vaso'] < vq[0])] = 1
discretised_int['vaso_input'][(interventions['max_dose_vaso'] >= vq[0]) & (interventions['max_dose_vaso'] < vq[1])] = 2
discretised_int['vaso_input'][(interventions['max_dose_vaso'] >= vq[1]) & (interventions['max_dose_vaso'] < vq[2])] = 3
a = interventions['max_dose_vaso'] >= vq[2]
discretised_int['vaso_input'][a] = 4

In [13]:
discretised_int['iv_input'] = discretised_int['input_4hourly']
discretised_int['iv_input'][interventions['input_4hourly'] == 0.0] = 0
discretised_int['iv_input'][(interventions['input_4hourly'] > 0.0) & (interventions['input_4hourly'] < ivq[0])] = 1
discretised_int['iv_input'][(interventions['input_4hourly'] >=  ivq[0]) & (interventions['input_4hourly'] <  ivq[1])] = 2
discretised_int['iv_input'][(interventions['input_4hourly'] >=  ivq[1]) & (interventions['input_4hourly'] < ivq[2])] = 3
discretised_int['iv_input'][(interventions['input_4hourly'] >=  ivq[2])] = 4

In [14]:
disc_inp_data = copy.deepcopy(orig_data)

In [15]:
disc_inp_data['vaso_input'] = discretised_int['vaso_input']
disc_inp_data['iv_input'] = discretised_int['iv_input']

In [20]:
actions = []
for i, row in disc_inp_data.iterrows():
    if row["vaso_input"] == 0:
        if row["iv_input"] == 0:
            actions.append(0)
        elif row["iv_input"] == 1:
            actions.append(5)
        elif row["iv_input"] == 2:
            actions.append(10)
        elif row["iv_input"] == 3:
            actions.append(15)
        else:
            actions.append(20)
    elif row["vaso_input"] == 1:
        if row["iv_input"] == 0:
            actions.append(1)
        elif row["iv_input"] == 1:
            actions.append(6)
        elif row["iv_input"] == 2:
            actions.append(11)
        elif row["iv_input"] == 3:
            actions.append(16)
        else:
            actions.append(21)
    elif row["vaso_input"] == 2:
        if row["iv_input"] == 0:
            actions.append(2)
        elif row["iv_input"] == 1:
            actions.append(7)
        elif row["iv_input"] == 2:
            actions.append(12)
        elif row["iv_input"] == 3:
            actions.append(17)
        else:
            actions.append(22)
    elif row["vaso_input"] == 3:
        if row["iv_input"] == 0:
            actions.append(3)
        elif row["iv_input"] == 1:
            actions.append(8)
        elif row["iv_input"] == 2:
            actions.append(13)
        elif row["iv_input"] == 3:
            actions.append(18)
        else:
            actions.append(23)
    else:
        if row["iv_input"] == 0:
            actions.append(4)
        elif row["iv_input"] == 1:
            actions.append(9)
        elif row["iv_input"] == 2:
            actions.append(14)
        elif row["iv_input"] == 3:
            actions.append(19)
        else:
            actions.append(24)
disc_inp_data["action"] = actions    

In [22]:
disc_inp_data.to_csv('../Fresh_start/discretised_input_data_MIMIC_mimiciii.csv', index=False)

Data saved and re-read so code can be paused here

In [ ]:
df = pd.read_csv('../Fresh_start/discretised_input_data_MIMIC_mimiciii.csv')

In [ ]:
c0 = -0.1/4
c1 = -0.5/4
c2 = -2

In [ ]:
#Add rewards
df["mortality_90d"] = df["mortality_90d"].fillna(0)
df['reward'] = 0
for i in df.index:
    if i == 0:
        continue
    # add intermediate rewards
    elif df.loc[i, 'icustayid'] == df.loc[i-1, 'icustayid']:
        sofa_cur = df.loc[i,'SOFA']
        sofa_prev = df.loc[i-1,'SOFA']
        lact_cur = df.loc[i,'Arterial_lactate']
        lact_prev = df.loc[i-1,'Arterial_lactate']
        reward = 0
        if sofa_cur == sofa_prev and sofa_cur != 0:
            reward += c0
        reward += c1*(sofa_cur-sofa_prev)
        reward += c2*np.tanh(lact_cur - lact_prev)
        df.loc[i-1,'reward'] = reward
    
    #Add final rewards
    elif df.loc[i, 'icustayid'] != df.loc[i-1, 'icustayid']:
        if df.loc[i-1, 'mortality_90d'] == 1.0:
                df.loc[i-1,'reward'] = -15
                
        elif df.loc[i-1, 'mortality_90d'] == 0.0:
            df.loc[i-1,'reward'] = 15
        else:
            print("error in row", i-1)
    elif i % 10000 == 0:
        print(i)
if df.loc[len(df)-1, 'mortality_90d'] == 1.0:
    df.loc[len(df)-1, 'reward'] = -15
elif df.loc[len(df)-1, 'mortality_90d'] == 0.0:
     df.loc[len(df)-1, 'reward'] = 15

In [ ]:
df.to_csv('../Fresh_start/did_MIMIC_mimiciii_with_rewards.csv',index=False)

Data saved and re-read so code can be paused here

In [ ]:
disc_inp_data = pd.read_csv('../Fresh_start/did_MIMIC_mimiciii_with_rewards.csv')

In [ ]:
# now split into train/validation/test sets
import random
unique_ids = disc_inp_data['icustayid'].unique()
random.shuffle(unique_ids)
train_sample = 0.7
val_sample = 0.1
test_sample = 0.2
train_num = int(len(unique_ids) * 0.7)
val_num = int(len(unique_ids)*0.1) + train_num
train_ids = unique_ids[:train_num]
val_ids = unique_ids[train_num:val_num]
test_ids = unique_ids[val_num:]

In [ ]:
train_set = DataFrame()
train_set = disc_inp_data.loc[disc_inp_data['icustayid'].isin(train_ids)]

val_set = DataFrame()
val_set = disc_inp_data.loc[disc_inp_data['icustayid'].isin(val_ids)]

test_set = DataFrame()
test_set = disc_inp_data.loc[disc_inp_data['icustayid'].isin(test_ids)]

In [ ]:
pregnant_set = pd.read_csv("../Matlab/pregnant_data_nn.csv")
control_1_set = pd.read_csv("../Matlab/control_1_data_nn.csv")

In [ ]:
pregnant_ids = pregnant_set["icustayid"]
control_1_ids = control_1_set["icustayid"]

In [ ]:
binary_fields = ['gender','mechvent','re_admission']
norm_fields= ['age','Weight_kg','GCS','HR','SysBP','MeanBP','DiaBP','RR','Temp_C','FiO2_1',
    'Potassium','Sodium','Chloride','Glucose','Magnesium','Calcium',
    'Hb','WBC_count','Platelets_count','PTT','PT','Arterial_pH','paO2','paCO2',
    'Arterial_BE','HCO3','Arterial_lactate','SOFA','SIRS','Shock_Index',
    'PaO2_FiO2','cumulated_balance', 'elixhauser', 'Albumin', u'CO2_mEqL', 'Ionised_Ca']
log_fields = ['max_dose_vaso','SpO2','BUN','Creatinine','SGOT','SGPT','Total_bili','INR',
              'input_total','input_4hourly','output_total','output_4hourly', 'bloc']

In [ ]:
train_set[binary_fields] = train_set[binary_fields] - 0.5 
val_set[binary_fields] = val_set[binary_fields] - 0.5 
test_set[binary_fields] = test_set[binary_fields] - 0.5
pregnant_set[binary_fields] = pregnant_set[binary_fields] - 0.5
control_1_set[binary_fields] = control_1_set[binary_fields] - 0.5

In [ ]:
# normal distn fields
for item in norm_fields:
    av = train_set[item].mean()
    #av = disc_inp_data[item].mean()
    std = train_set[item].std()
    #std = disc_inp_data[item].std()
    train_set[item] = (train_set[item] - av) / std
    val_set[item] = (val_set[item] - av) / std
    test_set[item] = (test_set[item] - av) / std
    pregnant_set[item] = (pregnant_set[item] - av) / std
    control_1_set[item] = (control_1_set[item] - av) / std

In [ ]:
# log normal fields
train_set[log_fields] = np.log(0.1 + train_set[log_fields])
val_set[log_fields] = np.log(0.1 + val_set[log_fields])
test_set[log_fields] = np.log(0.1 + test_set[log_fields])
pregnant_set[log_fields] = np.log(0.1 + pregnant_set[log_fields])
control_1_set[log_fields] = np.log(0.1 + control_1_set[log_fields])
for item in log_fields:
    av = train_set[item].mean()
    #av = disc_inp_data[item].mean()
    std = train_set[item].std()
    #std = disc_inp_data[item].std()
    train_set[item] = (train_set[item] - av) / std
    val_set[item] = (val_set[item] - av) / std
    test_set[item] = (test_set[item] - av) / std
    pregnant_set[item] = (pregnant_set[item] - av) / std
    control_1_set[item] = (control_1_set[item] - av) / std

In [ ]:
train_set.to_csv('../Fresh_start/rl_train_set_unscaled_rewarded.csv',index = False)
val_set.to_csv('../Fresh_start/rl_val_set_unscaled_rewarded.csv', index = False)
test_set.to_csv('../Fresh_start/rl_test_set_unscaled_rewarded.csv', index = False)
pregnant_set.to_csv('../Fresh_start/rl_pregnant_set_unscaled_rewarded.csv', index = False)
control_1_set.to_csv('../Fresh_start/rl_control_1_set_unscaled_rewarded.csv', index = False)

In [ ]:
# scale features to [0,1] in train set, similar in val and test
import copy
scalable_fields = copy.deepcopy(binary_fields)
scalable_fields.extend(norm_fields)
scalable_fields.extend(log_fields)
for col in scalable_fields:
    minimum = min(train_set[col])
    #minimum = min(disc_inp_data[col])
    maximum = max(train_set[col])
    #maximum = max(disc_inp_data[col])
    train_set[col] = (train_set[col] - minimum)/(maximum-minimum)
    val_set[col] = (val_set[col] - minimum)/(maximum-minimum)
    test_set[col] = (test_set[col] - minimum)/(maximum-minimum)
    pregnant_set[col] = (pregnant_set[col] - minimum)/(maximum-minimum)
    control_1_set[col] = (control_1_set[col] - minimum)/(maximum-minimum)

In [ ]:
train_set.to_csv('../Fresh_start/rl_train_set_scaled_rewarded.csv',index = False)
val_set.to_csv('../Fresh_start/rl_val_set_scaled_rewarded.csv', index = False)
test_set.to_csv('../Fresh_start/rl_test_set_scaled_rewarded.csv', index = False)
pregnant_set.to_csv('../Fresh_start/rl_pregnant_set_scaled_rewarded.csv', index = False)
control_1_set.to_csv('../Fresh_start/rl_control_1_set_scaled_rewarded.csv', index = False)

In [ ]:
train_set["max_dose_vaso_unnorm"] = disc_inp_data["max_dose_vaso"].loc[disc_inp_data['icustayid'].isin(train_ids)]
val_set["max_dose_vaso_unnorm"] = disc_inp_data["max_dose_vaso"].loc[disc_inp_data['icustayid'].isin(val_ids)]
test_set["max_dose_vaso_unnorm"] = disc_inp_data["max_dose_vaso"].loc[disc_inp_data['icustayid'].isin(test_ids)]
pregnant_set["max_dose_vaso_unnorm"] = disc_inp_data["max_dose_vaso"].loc[disc_inp_data['icustayid'].isin(pregnant_ids)].reset_index()["max_dose_vaso"]
control_1_set["max_dose_vaso_unnorm"] = disc_inp_data["max_dose_vaso"].loc[disc_inp_data['icustayid'].isin(control_1_ids)].reset_index()["max_dose_vaso"]

In [ ]:
def add_abchange_vc(df):
    abchange_vc_list = [0]
    for i, row in df.iterrows():
        if i == 0:
            prev_vc = row["max_dose_vaso_unnorm"]
        else:
            #print("eyyy")
            abchange_vc = row["max_dose_vaso_unnorm"] - prev_vc
            abchange_vc_list.append(abchange_vc)
            prev_vc = row["max_dose_vaso_unnorm"]
            #if i == 100:
                #break
    df.head()
    df["abchange_vc"] = abchange_vc_list
    return df

In [ ]:
train_set = train_set.reset_index()
train_set = add_abchange_vc(train_set)
test_set = test_set.reset_index()
val_set = val_set.reset_index()
test_set = add_abchange_vc(test_set)
val_set = add_abchange_vc(val_set)
pregnant_set = pregnant_set.reset_index()
pregnant_set = add_abchange_vc(pregnant_set)
control_1_set = control_1_set.reset_index()
control_1_set = add_abchange_vc(control_1_set)

In [ ]:
train_set.to_csv('../Fresh_start/rl_train_set_scaled_rewarded_abchange.csv',index = False)
val_set.to_csv('../Fresh_start/rl_val_set_scaled_rewarded_abchange.csv', index = False)
test_set.to_csv('../Fresh_start/rl_test_set_scaled_rewarded_abchange.csv', index = False)
pregnant_set.to_csv('../Fresh_start/rl_pregnant_set_scaled_rewarded_abchange.csv', index = False)
control_1_set.to_csv('../Fresh_start/rl_control_1_set_scaled_rewarded_abchange.csv', index = False)